In [454]:
! pip install pandas
! pip install openpyxl
! pip install xlsxwriter
import pandas as pd
import numpy as np

In [455]:
data = pd.ExcelFile("Rearranged_Sheet.xlsx")
sheet_names = data.sheet_names
sheet_names

['PROJECT',
 'OUTPUT 1',
 'OUTPUT 2',
 'OUTPUT 3',
 'OUTPUT 4',
 'OUTPUT 5',
 'OUTPUT 6',
 'OUTPUT 7',
 'INDICATOR 1',
 'INDICATOR 2',
 'INDICATOR 3',
 'INDICATOR 4',
 'INDICATOR 5',
 'INDICATOR 6',
 'INDICATOR 7']

In [456]:
project = data.parse('PROJECT', dtype={'DONOR CODE': object}).copy()
project

,PROJECT_TITLE,PROJECT_ID,PROJECT_DESCRIPTION,TOTAL_REQUIRED_RESOURCE,TOTAL_AVAILABLE_RESOURCE,RESOURCE_TYPE,DONOR_NAME,DONOR_CODE,AMOUNT,COMMENT,CLEARANCE_ROLE,NAME,APPROVAL_DATE,BOARD_DATE
0,Engagement facility West and Central Africa an...,00092084 / 00117764 / 00126438,NaN,442 398.5,442 398.5,CORE,NaN,NaN,0.0,Expected contribution of 86240 USD early 2025 ...,Project Manager,David Larue,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,DONOR,UNFPA,12445.0,111240.0,NaN,Director/Deputy Director for the Practice area,Henri Dommel,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,DONOR,LUXEMBOURG,155.0,305696.5,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,DONOR,UNDP,12.0,25462.0,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,IN KIND,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Host Government or stakeholder,NaN,NaN,NaN


In [457]:
project_columns = project.columns
project_columns

Index(['PROJECT_TITLE', 'PROJECT_ID', 'PROJECT_DESCRIPTION',
       'TOTAL_REQUIRED_RESOURCE', 'TOTAL_AVAILABLE_RESOURCE', 'RESOURCE_TYPE',
       'DONOR_NAME', 'DONOR_CODE', 'AMOUNT', 'COMMENT', 'CLEARANCE_ROLE',
       'NAME', 'APPROVAL_DATE', 'BOARD_DATE'],
      dtype='object')

In [458]:
def filterList(item_list, index_list):
    new_list = []
    for index in index_list:
        new_list.append(item_list[index])
        
    return new_list

project_table_column = filterList(project_columns, [0,1,2,3,4,9,12])
project_table_column
       

['PROJECT_TITLE',
 'PROJECT_ID',
 'PROJECT_DESCRIPTION',
 'TOTAL_REQUIRED_RESOURCE',
 'TOTAL_AVAILABLE_RESOURCE',
 'COMMENT',
 'APPROVAL_DATE']

In [459]:
project_table = project[project_table_column]

project_table = project_table.dropna(subset=['PROJECT_ID'])
this_project_id = project_table.iloc[0]['PROJECT_ID']
project_table

,PROJECT_TITLE,PROJECT_ID,PROJECT_DESCRIPTION,TOTAL_REQUIRED_RESOURCE,TOTAL_AVAILABLE_RESOURCE,COMMENT,APPROVAL_DATE
0,Engagement facility West and Central Africa an...,00092084 / 00117764 / 00126438,NaN,442 398.5,442 398.5,Expected contribution of 86240 USD early 2025 ...,NaN


In [460]:
donor_table = project[['DONOR_NAME', 'DONOR_CODE']]
donor_table = donor_table.dropna()
donor_table

,DONOR_NAME,DONOR_CODE
1,UNFPA,12445.0
2,LUXEMBOURG,155.0
3,UNDP,12.0


In [461]:
project_resources_table = project[filterList(project_columns, [5,7,8])]
project_resources_table = project_resources_table.dropna(subset=[project_columns[5]])
project_resources_table['PROJECT_ID'] = this_project_id
project_resources_table

,RESOURCE_TYPE,DONOR_CODE,AMOUNT,PROJECT_ID
0,CORE,NaN,0.0,00092084 / 00117764 / 00126438
1,DONOR,12445.0,111240.0,00092084 / 00117764 / 00126438
2,DONOR,155.0,305696.5,00092084 / 00117764 / 00126438
3,DONOR,12.0,25462.0,00092084 / 00117764 / 00126438
4,IN KIND,NaN,0.0,00092084 / 00117764 / 00126438


In [462]:
project_post_title_table = project[filterList(project_columns, [10, 11])]
project_post_title_table = project_post_title_table.dropna()
project_post_title_table = project_post_title_table.rename(columns={'CLEARANCE_ROLE': 'POST_TITLE'})
project_post_title_table['PROJECT_ID'] = this_project_id
project_post_title_table

,POST_TITLE,NAME,PROJECT_ID
0,Project Manager,David Larue,00092084 / 00117764 / 00126438
1,Director/Deputy Director for the Practice area,Henri Dommel,00092084 / 00117764 / 00126438


In [463]:
project_approval_table = project[['CLEARANCE_ROLE', 'NAME', 'APPROVAL_DATE']]
project_approval_table = project_approval_table.dropna(subset=['CLEARANCE_ROLE'])
project_approval_table['PROJECT_ID'] = this_project_id
project_approval_table

,CLEARANCE_ROLE,NAME,APPROVAL_DATE,PROJECT_ID
0,Project Manager,David Larue,NaN,00092084 / 00117764 / 00126438
1,Director/Deputy Director for the Practice area,Henri Dommel,NaN,00092084 / 00117764 / 00126438
5,Host Government or stakeholder,NaN,NaN,00092084 / 00117764 / 00126438


In [464]:
# The tables to be created with each sheets
budget_code_table = pd.DataFrame(columns=['CODE', 'DESCRIPTION'])
activity_budget_table = pd.DataFrame(columns=['PLANNED_ACTIVITY', 'BUDGET_CODE'])
output_table = pd.DataFrame(columns=['PROJECT_ID', 'PROJECT_OUTCOME', 'PROJECT_OUTPUT'])
indicator_table = pd.DataFrame(columns=[
 'PROJECT_OUTPUT',
 'INDICATOR',
 'DISAGGREGATION',
 'BASELINES',
 'YEAR',
 'ANNUAL_TARGET',
 'UNIT',
 'MID_YEAR_ACTUALS',
 'END_YEAR_ACTUALS',
 'DATA_SOURCE'])

activity_columns = [
    'PROJECT_OUTPUT',
    'PLANNED_ACTIVITY',
    'Q1',
    'Q2',
    'Q3',
    'Q4',
    'RESPONSIBLE_PARTY',
    'DONOR_CODE',
    'ANNUAL_BUDGET',
    'AMOUNT_FUNDED',
    'AMOUNT_UNFUNDED',
    'Q1_EXPENDITURE',
    'Q2_EXPENDITURE',
    'Q3_EXPENDITURE',
    'Q4_EXPENDITURE',
    'PROGRESS'
]
planned_activity_table = pd.DataFrame(columns=activity_columns)
activity_country_table = pd.DataFrame(columns=['PLANNED_ACTIVITY', 'COUNTRY'])


In [465]:
count = 1
# for each output and indicator sheets insert data
while True:
    try:
        act =data.parse(f'OUTPUT {count}', header=2)
        
        # adding to the output table
        oc = data.parse(f'OUTPUT {count}').iloc[0:1]
        oc['PROJECT_ID'] = this_project_id
        oc = oc[output_table.columns.to_list()]
        output_table = pd.concat([output_table, oc], ignore_index=True)
        output_table = output_table.drop_duplicates()
        
        for index, row in act.iterrows():
            # add to budget code table
            if not pd.isna(row['BUDGET_DESCRIPTION']):
                for description in row['BUDGET_DESCRIPTION'].split(','):
                    code = description.split('-')[0].strip()
                    desc =  description.split('-')[1].strip().lower().capitalize()
                    
                    budget_concat = pd.DataFrame([[code, desc]], columns=['CODE', 'DESCRIPTION'])
                    activity_budget_concat = pd.DataFrame([[row['PLANNED_ACTIVITY'], code]], columns=['PLANNED_ACTIVITY', 'BUDGET_CODE'])
                    
                    budget_code_table = pd.concat([budget_code_table, budget_concat])
                    budget_code_table = budget_code_table.drop_duplicates()
                    
                    activity_budget_table = pd.concat([activity_budget_table, activity_budget_concat], ignore_index=True)
            else:
                activity_budget_concat = pd.DataFrame([[row['PLANNED_ACTIVITY'], row['BUDGET_DESCRIPTION']]], columns=['PLANNED_ACTIVITY', 'BUDGET_CODE'])
                
                activity_budget_table = pd.concat([activity_budget_table, activity_budget_concat], ignore_index=True)
                
            for country in row['COUNTRY'].split(','):
                country_name = country.lower().capitalize()
                country_concat = pd.DataFrame([[row['PLANNED_ACTIVITY'],country_name]], columns=activity_country_table.columns.to_list())
                activity_country_table = pd.concat([activity_country_table, country_concat], ignore_index=True)
                
            activity_country_table = activity_country_table.drop_duplicates()
            activity_budget_table = activity_budget_table.drop_duplicates()
            
        act['PROJECT_OUTPUT'] = oc.iloc[0]['PROJECT_OUTPUT']
        newdf = act[activity_columns]
        planned_activity_table = pd.concat([planned_activity_table, newdf], ignore_index=True)
        planned_activity_table = planned_activity_table.drop_duplicates()


        
    except ValueError as e:
        print(e)
        break
    
    try:
        # adding to indicator table
        ind = data.parse(f'INDICATOR {count}')
        ind['PROJECT_OUTPUT'] = oc.iloc[0]['PROJECT_OUTPUT']
        ind = ind[indicator_table.columns.to_list()]
        indicator_table = pd.concat([indicator_table, ind], ignore_index=True)
        indicator_table = indicator_table.drop_duplicates()
    except ValueError as e:
        print(e)
        pass
    
    count += 1
    
activity_country_table

/var/folders/js/dwxqc6wn6j9dlq3v4hscvggm0000gn/T/ipykernel_84026/372260259.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  planned_activity_table = pd.concat([planned_activity_table, newdf], ignore_index=True)


Worksheet named 'OUTPUT 8' not found


,PLANNED_ACTIVITY,COUNTRY
0,Activity 1.1. Conduct a training campaign on t...,Burkina faso
1,Activity 2.1 Support for the structuring of yo...,Burkina faso
2,Activty 2.2:Organizing a awarness day on finan...,Burkina faso
3,Actvitiy 3.1.Facilitate the development of fin...,Burkina faso
4,Activity 3.2 Open accounts with financial serv...,Burkina faso
5,Actvitiy 4.1. Coaching of young people for the...,Burkina faso
6,Activity 4.2 Connect with MFIs,Burkina faso
7,Actvitiy 6.1 One monitoring visit and one capi...,Burkina faso
8,Actvitiy 6.2 : Quaterly and final reporting,Burkina faso
9,Actvitiy 6.3 : Coordination gestion,Burkina faso



LIST OF SHEETS/ TABLES
- project_table
- donor_table
- project_resource_table
- project_post_title_table
- project_approval_table
- output_table
- planned_activity_table
- activity_country_table
- budget_code_table
- activity_budget_table
- indicator_table


In [466]:
with pd.ExcelWriter('NORMALIZED_TABLE.xlsx', engine='xlsxwriter') as writer:
    project_table.to_excel(writer, sheet_name='PROJECT')
    donor_table.to_excel(writer, sheet_name='DONOR')
    project_resources_table.to_excel(writer, sheet_name='PROJECT_RESOURCE')
    project_post_title_table.to_excel(writer, sheet_name='PROJECT_POST_TITLE')
    project_approval_table.to_excel(writer, sheet_name='PROJECT_APPROVAL')
    output_table.to_excel(writer, sheet_name='OUTPUT')
    planned_activity_table.to_excel(writer, sheet_name='PLANNED_ACTIVITY')
    activity_country_table.to_excel(writer, sheet_name='ACTIVITY_COUNTRY')
    budget_code_table.to_excel(writer, sheet_name='BUDGET_CODE')
    activity_budget_table.to_excel(writer, sheet_name='ACTIVITY_BUDGET')
    indicator_table.to_excel(writer, sheet_name='INDICATOR')